In [ ]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "key.json"

In [ ]:
PROJECT_ID = "mlops-433612"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

API_ENDPOINT = f"{LOCATION}-aiplatform.googleapis.com"

from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
from google.cloud.aiplatform_v1beta1 import (
    FeatureOnlineStoreAdminServiceClient, FeatureRegistryServiceClient)

admin_client = FeatureOnlineStoreAdminServiceClient(
    client_options={"api_endpoint": API_ENDPOINT}
)
registry_client = FeatureRegistryServiceClient(
    client_options={"api_endpoint": API_ENDPOINT}
)

In [ ]:
from vertexai.resources.preview import FeatureOnlineStore

FEATURE_ONLINE_STORE_ID = "test_online_store"  # @param {type:"string"}

my_fos = FeatureOnlineStore.create_optimized_store(FEATURE_ONLINE_STORE_ID)

In [ ]:
from vertexai.resources.preview import FeatureViewBigQuerySource

FEATURE_VIEW_ID = "test_fv"  # @param {type:"string"}

CRON_SCHEDULE = "* * * * *"  # @param {type:"string"}

DATASET_ID = "featurestore_demo"
TABLE_ID = "cleaned_table"

BQ_VIEW_ID_FQN = f'{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}'

my_fv = my_fos.create_feature_view(
    FEATURE_VIEW_ID,
    FeatureViewBigQuerySource(
        uri=f"bq://{BQ_VIEW_ID_FQN}",
        entity_id_columns=["entity_id"],
    ),
    # sync_config can be set with CRON_SCHEDULE if you want to setup auto sync
    sync_config=None,
)

In [ ]:
fv_sync = my_fv.sync()

import time

while True:
    feature_view_sync = my_fv.get_sync(fv_sync.resource_name.split("/")[9]).gca_resource
    if feature_view_sync.run_time.end_time.seconds > 0:
        status = "Succeed" if feature_view_sync.final_status.code == 0 else "Failed"
        print(f"Sync {status} for {feature_view_sync.name}.")
        # wait a little more for the job to properly shutdown
        time.sleep(30)
        break
    else:
        print("Sync ongoing, waiting for 30 seconds.")
    time.sleep(30)

In [ ]:
from vertexai.resources.preview import FeatureView
import pandas as pd

# Depends on the FeatureView you created, the FEATURE_VIEW_ID here can be different
data = FeatureView(name=FEATURE_VIEW_ID, feature_online_store_id=FEATURE_ONLINE_STORE_ID).read(key=["10"]).to_dict()

print(data)